In [ ]:
import json
import os
import SharedArray as sa
from typing import Optional
from datetime import datetime
import json
import random
import numpy as np
import torch
import pandas as pd
import glob
from alphagen.data.expression import *
from alphagen_ocean.calculator import QLibStockDataCalculator
from alphagen_ocean.stock_data import ArgData
from alphagen.config import *
from alphagen.data.tree import ExpressionBuilder
from alphagen.data.tokens import *

from utils import formula_to_expression

In [ ]:
device = torch.device("cpu")
cuda = torch.device("cuda:3")
data_test = ArgData(
    start_time=20210101,
    end_time=20211231,
    device=device,
)

In [ ]:
from glob import glob
paths = sorted(glob('checkpoints/*/'))
json_paths = [sorted(
              glob(path + "*.json"), key=lambda x: int(x.split("/")[-1].split("_")[0])
            )[-1] for path in paths]

In [ ]:
import utils
from concurrent.futures import ProcessPoolExecutor

In [ ]:
with ProcessPoolExecutor(max_workers=160) as executor:
    results = executor.map(utils.json_to_factor, json_paths)
    

In [ ]:
factors = list(results)

In [ ]:
factors

In [ ]:
import ray
ray.init(num_cpus=160)

In [ ]:
n = len(factors)
futures = []

for i in range(n):
        for j in range(i + 1, n):
                future = utils.remote_batch_pearsonr.remote(factors[i], factors[j])
                futures.append(future)
df_corr = np.zeros((n, n), dtype=np.float32)
results = ray.get(futures)
for i in range(n):
        for j in range(i + 1, n):
                df_corr[i][j] = results[i + j]
                df_corr[j][i] = results[i + j]
df_corr = df_corr

In [ ]:
pd.DataFrame(df_corr).round(4)

In [ ]:
import modin.pandas as mpd

In [ ]:
featmat = torch.stack([factor.reshape(-1,1) for factor in factors],dim=1).squeeze()

In [ ]:
featmat.shape

In [ ]:
import pandas as pd
import numpy as np
from alphagen.utils.correlation import pd_corr_numba

In [ ]:
a = pd.DataFrame(np.random.randn(8532000, 114))

In [ ]:
pd_corr_numba(a)